In [1]:
!pip install datasets
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 39.9 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.1
    Uninstalling pyarrow-19.0.1:
      Successfully uninstalled pyarrow-19.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
cudf-polars-cu12 25.6.0 requires pylibcudf-cu12==25.6.*, but you have pylibcudf-cu12 25.2.2 which is incompatible.


In [1]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datasets
import torch
import wandb
from collections import Counter
from datasets import load_dataset, concatenate_datasets
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer, DataCollatorWithPadding
)
from peft import LoraConfig, get_peft_model, TaskType
from sklearn.metrics import accuracy_score, f1_score, classification_report
import os
import json

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [2]:
from datasets import load_dataset

ds = load_dataset("facebook/anli")

README.md: 0.00B [00:00, ?B/s]

plain_text/train_r1-00000-of-00001.parqu(…):   0%|          | 0.00/3.14M [00:00<?, ?B/s]

plain_text/dev_r1-00000-of-00001.parquet:   0%|          | 0.00/351k [00:00<?, ?B/s]

plain_text/test_r1-00000-of-00001.parque(…):   0%|          | 0.00/353k [00:00<?, ?B/s]

plain_text/train_r2-00000-of-00001.parqu(…):   0%|          | 0.00/6.53M [00:00<?, ?B/s]

...

Generating dev_r2 split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test_r2 split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating train_r3 split:   0%|          | 0/100459 [00:00<?, ? examples/s]

Generating dev_r3 split:   0%|          | 0/1200 [00:00<?, ? examples/s]

Generating test_r3 split:   0%|          | 0/1200 [00:00<?, ? examples/s]

In [7]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret("wandb-key")
wandb.login(key=api_key)
WANDB_PROJECT = "EAI-Assignment-Multi-class-classification-using-ANLI"

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
def load_combined_dataset(use_r1=True):
    dataset = load_dataset("facebook/anli")

    train_data = dataset["train_r2"]
    val_data = dataset["dev_r2"]
    test_data = dataset["test_r2"]

    print(f"R2 train: {len(train_data)} samples")

    if use_r1:
        r1_train = dataset["train_r1"]
        train_data = concatenate_datasets([train_data, r1_train])
        print(f"+ R1: {len(r1_train)} samples added")

    print(f"TOTAL training samples = {len(train_data)}")
    print(f"Label distribution: {Counter(train_data['label'])}")
    
    return train_data, val_data, test_data

In [7]:
OUTPUT_BASE = "/kaggle/working/ANLI_Experiments"
os.makedirs(OUTPUT_BASE, exist_ok=True)



## 1. Configuration

Centralized experiment settings using a Python dataclass. Handles all hyperparameters including model name, LoRA settings, learning rate, epochs, and dataset configuration. Automatically generates run names and sets sensible defaults based on whether LoRA is enabled.

In [ ]:
@dataclass
class ExperimentConfig:
    """Central configuration for experiment parameters."""
    model_name: str
    use_lora: bool = True
    lora_r: int = 16
    lora_alpha: int = 32
    lora_dropout: float = 0.1
    dataset_config: str = "r1_r2"
    learning_rate: Optional[float] = None
    num_epochs: int = 5
    max_length: int = 256
    run_name: Optional[str] = None
    tags: Optional[List[str]] = None
    seed: int = 42
    
    def __post_init__(self):
        if self.run_name is None:
            model_short = self.model_name.split("/")[-1]
            lora_str = f"lora-r{self.lora_r}" if self.use_lora else "full"
            self.run_name = f"{model_short}-{lora_str}-{self.dataset_config}"
        
        if self.learning_rate is None:
            self.learning_rate = 2e-4 if self.use_lora else 2e-5
        
        if self.tags is None:
            self.tags = ["ANLI", "NLI"]
    
    @property
    def batch_size(self) -> int:
        return 16 if self.use_lora else 8

In [ ]:
def setup_directories(output_base: str, run_name: str) -> Dict[str, str]:
    """Create and return experiment directory paths."""
    paths = {
        "output": f"{output_base}/{run_name}",
        "checkpoints": f"{output_base}/{run_name}/checkpoints",
        "final_model": f"{output_base}/{run_name}/final_model",
    }
    
    for path in paths.values():
        os.makedirs(path, exist_ok=True)
    
    return paths

## 2. Data Loading & Tokenization

Loads the NLI dataset and applies tokenization using the model's tokenizer. Processes premise-hypothesis pairs with truncation, converts labels, and removes unnecessary columns. Returns train, validation, and test splits ready for training.

In [ ]:
def load_and_tokenize_data(
    tokenizer,
    dataset_config: str,
    max_length: int = 256,
) -> Tuple[Any, Any, Any]:
    """Load dataset and apply tokenization."""
    
    use_r1 = "r1" in dataset_config
    train_data, val_data, test_data = load_combined_dataset(use_r1=use_r1)
    
    def tokenize_fn(examples):
        enc = tokenizer(
            examples["premise"],
            examples["hypothesis"],
            truncation=True,
            max_length=max_length,
        )
        enc["labels"] = examples["label"]
        return enc
    
    cols_remove = ["uid", "premise", "hypothesis", "reason"]
    
    train_tok = train_data.map(tokenize_fn, batched=True, remove_columns=cols_remove)
    val_tok = val_data.map(tokenize_fn, batched=True, remove_columns=cols_remove)
    test_tok = test_data.map(tokenize_fn, batched=True, remove_columns=cols_remove)
    
    return train_tok, val_tok, test_tok



## 3. Model Setup

Initializes the base transformer model for sequence classification. When LoRA is enabled, applies parameter-efficient adapters targeting attention layers (query, key, value). Automatically detects DeBERTa models and adjusts target module names accordingly.

In [ ]:
def load_base_model(model_name: str, num_labels: int = 3):
    """Load the base classification model."""
    return AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=num_labels,
    )


In [ ]:
def apply_lora(model, config: ExperimentConfig):
    """Apply LoRA adapters to the model."""
    if "deberta" in config.model_name.lower():
        target_modules = ["query_proj", "key_proj", "value_proj"]
    else:
        target_modules = ["query", "key", "value"]
    
    lora_config = LoraConfig(
        task_type=TaskType.SEQ_CLS,
        r=config.lora_r,
        lora_alpha=config.lora_alpha,
        lora_dropout=config.lora_dropout,
        target_modules=target_modules,
        modules_to_save=["classifier", "pooler"],
    )
    
    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()
    
    # Debug: verify classifier is trainable
    for name, param in model.named_parameters():
        if 'classifier' in name:
            print(f"  {name}: requires_grad={param.requires_grad}")
    
    return model


In [ ]:
def setup_model(config: ExperimentConfig):
    """Load model and optionally apply LoRA."""
    model = load_base_model(config.model_name)
    
    if config.use_lora:
        model = apply_lora(model, config)
    
    return model

## 4. W&B Initialization

Sets up Weights & Biases experiment tracking. Logs all configuration parameters, dataset sizes, and training hyperparameters. Enables real-time monitoring of training progress and metric visualization.

In [ ]:
def init_wandb(
    project: str,
    config: ExperimentConfig,
    train_size: int,
    val_size: int,
    test_size: int,
) -> None:
    """Initialize Weights & Biases tracking."""
    wandb.init(
        project=project,
        name=config.run_name,
        config={
            "model_name": config.model_name,
            "use_lora": config.use_lora,
            "lora_r": config.lora_r if config.use_lora else None,
            "lora_alpha": config.lora_alpha if config.use_lora else None,
            "dataset": config.dataset_config,
            "train_samples": train_size,
            "val_samples": val_size,
            "test_samples": test_size,
            "learning_rate": config.learning_rate,
            "batch_size": config.batch_size,
            "epochs": config.num_epochs,
            "max_length": config.max_length,
        },
        tags=config.tags,
        reinit=True,
    )


## 5. Training Configuration

Builds HuggingFace TrainingArguments with optimized settings. Configures mixed-precision training, gradient accumulation, warmup schedule, and checkpoint saving strategy. Sets up epoch-based evaluation with F1-macro as the optimization target.

In [ ]:
def create_training_args(config: ExperimentConfig, checkpoint_dir: str) -> TrainingArguments:
    """Create HuggingFace TrainingArguments."""
    return TrainingArguments(
        output_dir=checkpoint_dir,
        num_train_epochs=config.num_epochs,
        
        per_device_train_batch_size=config.batch_size,
        per_device_eval_batch_size=config.batch_size * 2,
        gradient_accumulation_steps=2,
        
        learning_rate=config.learning_rate,
        weight_decay=0.01,
        warmup_ratio=0.1,
        
        fp16=True,
        optim="adamw_torch",
        
        eval_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="f1_macro",
        greater_is_better=True,
        
        logging_steps=50,
        report_to="wandb",
        
        seed=config.seed,
    )



## 6. Metrics

Computes evaluation metrics during training and testing. Calculates accuracy and macro-averaged F1 score. Also prints prediction distribution for debugging class imbalance issues.

In [ ]:
def compute_metrics(eval_pred) -> Dict[str, float]:
    """Compute accuracy and macro F1 for evaluation."""
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    
    unique, counts = np.unique(preds, return_counts=True)
    pred_dist = dict(zip(unique.tolist(), counts.tolist()))
    print(f"  Pred distribution: {pred_dist}")
    
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1_macro": f1_score(labels, preds, average="macro"),
    }


def generate_classification_report(labels, preds) -> Tuple[Dict, str]:
    """Generate classification report dict and string."""
    target_names = ['Entailment', 'Neutral', 'Contradiction']
    
    report_dict = classification_report(
        labels, preds,
        target_names=target_names,
        output_dict=True
    )
    
    report_str = classification_report(
        labels, preds,
        target_names=target_names
    )
    
    return report_dict, report_str



## 7. Training & Evaluation

Executes the training loop and runs final evaluation on the test set. Tracks training time, generates predictions, and produces a detailed classification report with per-class precision, recall, and F1 scores.

In [ ]:
def create_trainer(
    model,
    training_args: TrainingArguments,
    train_dataset,
    val_dataset,
    tokenizer,
) -> Trainer:
    """Create and return the Trainer instance."""
    return Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        data_collator=DataCollatorWithPadding(tokenizer),
        compute_metrics=compute_metrics,
    )


def train_model(trainer: Trainer) -> float:
    """Train the model and return training time in minutes."""
    start = time.time()
    trainer.train()
    return (time.time() - start) / 60

In [ ]:
def evaluate_model(trainer: Trainer, test_dataset) -> Dict[str, Any]:
    """Run evaluation and return comprehensive results."""
    test_results = trainer.evaluate(test_dataset)
    
    predictions = trainer.predict(test_dataset)
    preds = np.argmax(predictions.predictions, axis=-1)
    labels = predictions.label_ids
    
    report_dict, report_str = generate_classification_report(labels, preds)
    
    print(f"\nTEST RESULTS → ACC={test_results['eval_accuracy']:.4f}, F1={test_results['eval_f1_macro']:.4f}")
    print("\nClassification Report:")
    print(report_str)
    
    return {
        "test_results": test_results,
        "predictions": preds,
        "labels": labels,
        "report": report_dict,
    }



## 8. Logging & Saving

Handles all persistence operations after training completes. Logs final metrics and confusion matrix to W&B, saves model weights and tokenizer locally, writes results JSON, and uploads the trained model as a W&B artifact for versioning.

In [ ]:
def log_final_metrics(eval_results: Dict, train_time: float) -> None:
    """Log final metrics to W&B."""
    report = eval_results["report"]
    test_results = eval_results["test_results"]
    
    wandb.log({
        "test/accuracy": test_results['eval_accuracy'],
        "test/f1_macro": test_results['eval_f1_macro'],
        "test/f1_entailment": report['Entailment']['f1-score'],
        "test/f1_neutral": report['Neutral']['f1-score'],
        "test/f1_contradiction": report['Contradiction']['f1-score'],
        "train_time_min": train_time,
    })
    
    wandb.log({
        "confusion_matrix": wandb.plot.confusion_matrix(
            y_true=eval_results["labels"],
            preds=eval_results["predictions"],
            class_names=['Entailment', 'Neutral', 'Contradiction']
        )
    })
    
def save_model_and_results(
    trainer: Trainer,
    tokenizer,
    paths: Dict[str, str],
    run_name: str,
    eval_results: Dict,
    train_time: float,
) -> None:
    """Save model, tokenizer, and results JSON."""
    print(f"\n Saving model to: {paths['final_model']}")
    
    trainer.save_model(paths['final_model'])
    tokenizer.save_pretrained(paths['final_model'])
    
    with open(f"{paths['output']}/results.json", 'w') as f:
        json.dump({
            "run_name": run_name,
            "test_accuracy": eval_results["test_results"]['eval_accuracy'],
            "test_f1_macro": eval_results["test_results"]['eval_f1_macro'],
            "train_time_min": train_time,
        }, f, indent=2)


def upload_wandb_artifact(
    paths: Dict[str, str],
    run_name: str,
    model_name: str,
    use_lora: bool,
    eval_results: Dict,
    dataset_config: str,
) -> None:
    """Upload model to W&B Artifacts."""
    print(f"\n Uploading model to W&B Artifacts...")
    
    test_results = eval_results["test_results"]
    
    artifact = wandb.Artifact(
        name=f"model-{run_name}",
        type="model",
        description=f"Fine-tuned {model_name} on ANLI {dataset_config}",
        metadata={
            "accuracy": test_results['eval_accuracy'],
            "f1_macro": test_results['eval_f1_macro'],
            "model_name": model_name,
            "use_lora": use_lora,
        }
    )
    
    artifact.add_dir(paths['final_model'])
    wandb.log_artifact(artifact)
    
    print(f" Model uploaded to W&B Artifacts: model-{run_name}")



## 9. Main Orchestrator

The primary entry point that coordinates all modules in sequence. Takes experiment parameters, instantiates the config, and executes the full pipeline from data loading through model upload. Returns a summary dictionary with final metrics.

In [ ]:
def run_experiment(
    model_name: str,
    output_base: str,
    wandb_project: str,
    use_lora: bool = True,
    lora_r: int = 16,
    lora_alpha: int = 32,
    dataset_config: str = "r1_r2",
    learning_rate: Optional[float] = None,
    num_epochs: int = 5,
    run_name: Optional[str] = None,
    tags: Optional[List[str]] = None,
) -> Dict[str, Any]:
    """
    Main experiment orchestrator - coordinates all components.
    """
    # Create config
    config = ExperimentConfig(
        model_name=model_name,
        use_lora=use_lora,
        lora_r=lora_r,
        lora_alpha=lora_alpha,
        dataset_config=dataset_config,
        learning_rate=learning_rate,
        num_epochs=num_epochs,
        run_name=run_name,
        tags=tags,
    )
    
    print(f"\n{'='*60}")
    print(f" Starting Run: {config.run_name}")
    print(f"{'='*60}")
    
    # Setup directories
    paths = setup_directories(output_base, config.run_name)
    
    # Load tokenizer and data
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    train_tok, val_tok, test_tok = load_and_tokenize_data(
        tokenizer, config.dataset_config, config.max_length
    )
    
    # Setup model
    model = setup_model(config)
    
    # Init W&B
    init_wandb(wandb_project, config, len(train_tok), len(val_tok), len(test_tok))
    
    # Create training args and trainer
    training_args = create_training_args(config, paths["checkpoints"])
    trainer = create_trainer(model, training_args, train_tok, val_tok, tokenizer)
    
    # Train
    train_time = train_model(trainer)
    
    # Evaluate
    eval_results = evaluate_model(trainer, test_tok)
    
    #  Log, save, upload
    log_final_metrics(eval_results, train_time)
    save_model_and_results(trainer, tokenizer, paths, config.run_name, eval_results, train_time)
    upload_wandb_artifact(paths, config.run_name, model_name, use_lora, eval_results, config.dataset_config)
    
    # Cleanup
    wandb.finish()
    
    return {
        "run_name": config.run_name,
        "accuracy": eval_results["test_results"]['eval_accuracy'],
        "f1_macro": eval_results["test_results"]['eval_f1_macro'],
        "train_time": train_time,
    }

In [8]:
WANDB_RUN_NAME = "deberta-base-lora-r16-r2"


results = run_experiment(
    model_name="microsoft/deberta-v3-base",
    use_lora=True,
    lora_r=16,
    dataset_config="r2",
    run_name="deberta-base-lora-r16-r2",
    tags=["DeBERTa", "LoRA", "R2"],
)

print(f"\nFINAL RESULTS: {results}")


 Starting Run: deberta-base-lora-r16-r2
R2 train: 45460 samples
TOTAL training samples = 45460
Label distribution: Counter({1: 20959, 0: 14448, 2: 10053})


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


...

  Pred distribution: {0: 410, 1: 331, 2: 259}

TEST RESULTS → ACC=0.5030, F1=0.4992

Classification Report:
               precision    recall  f1-score   support

   Entailment       0.50      0.61      0.55       334
      Neutral       0.51      0.50      0.51       333
Contradiction       0.51      0.40      0.45       333

     accuracy                           0.50      1000
    macro avg       0.50      0.50      0.50      1000
 weighted avg       0.50      0.50      0.50      1000


 Saving model to: /kaggle/working/ANLI_Experiments/deberta-base-lora-r16-r2/final_model


wandb: Adding directory to artifact (/kaggle/working/ANLI_Experiments/deberta-base-lora-r16-r2/final_model)... Done. 0.0s



 Uploading model to W&B Artifacts...
 Model uploaded to W&B Artifacts: model-deberta-base-lora-r16-r2


eval/accuracy,▁▂▅▆▆█
eval/f1_macro,▁▂▆▆▆█
eval/loss,▃▄▁▄▇█
eval/runtime,▁█▄▁▁▃
eval/samples_per_second,█▁▅██▆
eval/steps_per_second,█▁▅██▆
test/accuracy,▁▁
test/f1_contradiction,▁
test/f1_entailment,▁
test/f1_macro,▁▁
test/f1_neutral,▁



FINAL RESULTS: {'run_name': 'deberta-base-lora-r16-r2', 'accuracy': 0.503, 'f1_macro': 0.4992229890208902, 'train_time': 61.8003990372022}


In [5]:
WANDB_RUN_NAME = "roberta-large-lora-r16-r2"

In [8]:

OUTPUT_BASE = "/kaggle/working/ANLI_Experiments"
os.makedirs(OUTPUT_BASE, exist_ok=True)



results = run_experiment(
    model_name="FacebookAI/roberta-large",
    use_lora=True,
    lora_r=16,
    num_epochs=4,
    dataset_config="r2",
    run_name="roberta-large-lora-r16-r2",
    tags=["RoBERTa", "LoRA", "R2"],
)

print(f"\nFINAL RESULTS: {results}")


 Starting Run: roberta-large-lora-r16-r2
R2 train: 45460 samples
TOTAL training samples = 45460
Label distribution: Counter({1: 20959, 0: 14448, 2: 10053})


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 3,411,971 || all params: 358,774,790 || trainable%: 0.9510
  base_model.model.classifier.original_module.dense.weight: requires_grad=False
  base_model.model.classifier.original_module.dense.bias: requires_grad=False
  base_model.model.classifier.original_module.out_proj.weight: requires_grad=False
  base_model.model.classifier.original_module.out_proj.bias: requires_grad=False
  base_model.model.classifier.modules_to_save.default.dense.weight: requires_grad=True
  base_model.model.classifier.modules_to_save.default.dense.bias: requires_grad=True
  base_model.model.classifier.modules_to_save.default.out_proj.weight: requires_grad=True
  base_model.model.classifier.modules_to_save.default.out_proj.bias: requires_grad=True


wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


...

  Pred distribution: {0: 425, 1: 329, 2: 246}

TEST RESULTS → ACC=0.4210, F1=0.4145

Classification Report:
               precision    recall  f1-score   support

   Entailment       0.42      0.53      0.47       334
      Neutral       0.46      0.46      0.46       333
Contradiction       0.37      0.28      0.32       333

     accuracy                           0.42      1000
    macro avg       0.42      0.42      0.41      1000
 weighted avg       0.42      0.42      0.41      1000


 Saving model to: /kaggle/working/ANLI_Experiments/roberta-large-lora-r16-r2/final_model


wandb: Adding directory to artifact (/kaggle/working/ANLI_Experiments/roberta-large-lora-r16-r2/final_model)... Done. 0.0s



 Uploading model to W&B Artifacts...
 Model uploaded to W&B Artifacts: model-roberta-large-lora-r16-r2


eval/accuracy,▁▅▆▅█
eval/f1_macro,▁▆▇▆█
eval/loss,▂▁▂█▂
eval/runtime,▁▅▁▅█
eval/samples_per_second,█▄█▃▁
eval/steps_per_second,█▄█▃▁
test/accuracy,▁▁
test/f1_contradiction,▁
test/f1_entailment,▁
test/f1_macro,▁▁
test/f1_neutral,▁



FINAL RESULTS: {'run_name': 'roberta-large-lora-r16-r2', 'accuracy': 0.421, 'f1_macro': 0.4144689851494887, 'train_time': 131.99350779453914}
